In [1]:
# development code
%load_ext autoreload
%autoreload 2

### Run MAF on Multiple FBS_2.0 OpSims Using `rubin_sim`

The previous example showed how to run a metric analysis on one or two opSims.  But what you really want to do is to be able to run through *all* of them.  This notebook shows how to do that.  The next notebook will demonstrate how to analyze the results.

### Software Setup (repeat each session)

In [2]:
# import common python packages
%matplotlib inline
import matplotlib.pyplot as plt
import os, sys

##### Detect and Set Your SciServer Username
__Note:__ Your usename will be used to store MAF output, so please make sure it is correct.

In [3]:
your_username = os.getcwd().split('/')[5]
print(f'Your automatically extracted username is: {your_username}.'
    ' If it is incorrect, please mannually reset it.')

Your automatically extracted username is: ywx649999311. If it is incorrect, please mannually reset it.


##### Import the $\texttt{rubin_sim}$ modules needed.

In [4]:
# import rubin_sim python modules
import rubin_sim.maf.db as db
import rubin_sim.maf.metrics as metrics
import rubin_sim.maf.slicers as slicers
import rubin_sim.maf.stackers as stackers
import rubin_sim.maf.plots as plots
import rubin_sim.maf.metricBundles as metricBundles

# import convenience functions for MAF analysis on SciServer
from opsimUtils import *

# print version
import rubin_sim
rubin_sim.__version__

'0.10.1.dev62+gbb0801e'

### 1. Build connections to the OpSims databases
The first step is to initiate opsim database objects and result database objects for the opsim databases that you want to run metrics on. Two paths are needed here:
1. `dbDir`: The path to the shared SciServer directory that host all of the data that you need to run MAF. 
2. `outDir`: The path to your own directory where you want to save the metric metadata.

<!-- **NOTE:** Opsims simulated using different versions of the Feature-based schedulers are stored in seperate folders, you can get the correct path using function `show_fbs_dirs()`. -->

In [38]:
dbRuns = list(run_df.index)
dbRuns[0:5] # only show first 5 opsims

['baseline_retrofoot_v2.0_10yrs',
 'baseline_v2.0_10yrs',
 'baseline_v2.1_10yrs',
 'bluer_indx0_v2.0_10yrs',
 'bluer_indx1_v2.0_10yrs']

In [19]:
dbDir = rubin_sim.data.get_data_dir()
outDir = '/home/idies/workspace/Storage/{}/persistent/MAFOutput'.format(your_username)

if not os.path.exists(os.path.abspath(outDir)):
    os.mkdir(os.path.abspath(outDir))

In [31]:
# two dictionary are returned by the following function, 
# One (opSimDbs) is a dictionary storing all database objects
# Another (resultDbs) is a dictionary consist of the objects directing MAF where to save metric metadata
# Both dictionaries are indexed by OpSim run names
opSimDbs, resultDbs = connect_dbs(dbDir, outDir, dbRuns=dbRuns[0:5])

You can use `help` to get more information about the provided convenience functions, e.g., `connect_dbs()`

In [8]:
help(connect_dbs)

Help on function connect_dbs in module opsimUtils:

connect_dbs(dbDir, outDir, dbRuns=None)
    Initiate database objects to all opSim databases in the provided directory.
    Returns a dictionary consisting all database connections and a dictionary
    holding the resultsDb objects.
    
    Args:
        dbDir(str): The path to the dabase directory.
        outDir(str): The path to the result database directory.
        dbRuns(list): A list of OpSim runs to connect to.
    
    Returns:
        opSimDbs(dict): A dictionary containing the OpsimDatabase objects for
            opsim databases in the provided directory, keys are the run names.
        resultDbs(str): A dictionary containing the ResultsDb objects for opsim
            databases in the provided directory, keys are the run names.



You can also check what OpSims are available in the directory. We suggest saving the returned list to a variable, which will make your life easier if you don't want to run metric evaluations on all avaiable opsims. For more options to select your runs (e.g., by family), please see the [Getting_Data](../rubin_sim_notebooks/maf_tutorial/04_Getting_Data.ipynb) notebook. 

#### 2. Declare some metrics to run on above OpSims

In [32]:
# Airmass Metric
maxMetric = metrics.MaxMetric('airmass')
airmassSlicer = slicers.HealpixSlicer(nside=64)
airmassConstraint = 'filter = "g"'
airmassConstraint += ' and note not like "DD%"'

maxairmassSky = metricBundles.MetricBundle(maxMetric, airmassSlicer, \
                airmassConstraint)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes


In [33]:
# Coadd Metric in Y band
coaddMetric = metrics.Coaddm5Metric()
coaddSlicer = slicers.HealpixSlicer(nside=64)
coaddConstraint = 'filter = "y"'
coaddConstraint += ' and note not like "DD%"'

CoaddY = metricBundles.MetricBundle(coaddMetric, coaddSlicer, \
                coaddConstraint)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes


In [34]:
# Coadd Metric in U band
coaddMetric = metrics.Coaddm5Metric()
coaddSlicer = slicers.HealpixSlicer(nside=64)
coaddConstraintU = 'filter = "u"'
coaddConstraintU += ' and note not like "DD%"'

CoaddU = metricBundles.MetricBundle(coaddMetric, coaddSlicer, \
                coaddConstraintU)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes


In [35]:
# set summary statistics
summaryMetrics = [metrics.MinMetric(), metrics.MedianMetric(),
                  metrics.MaxMetric(), metrics.RmsMetric()]
maxairmassSky.setSummaryMetrics(summaryMetrics)
CoaddY.setSummaryMetrics(summaryMetrics)
CoaddU.setSummaryMetrics(summaryMetrics)

In [36]:
# create a bundleDict for all metrics to run on each opSim
bundleDict = {'maxairmassSky': maxairmassSky, 'CoaddY': CoaddY, 'CoaddU': CoaddU}

#### 3. Loop over all OpSims in dbDir and run MAF
While constructing a metricBundleGroup from a dictionary (the cell below), you will need to provide the path to a directory (`metricDataPath` in the cell below) where you would like to store the metric data (this is **DIFFERENT** than path to the metric data, `outDir`). To construct metricbundles for plotting and further analysis, this path will be needed.

In [37]:
metricDataPath = '/home/idies/workspace/Storage/{}/persistent/MAFOutput/MetricData/'.format(your_username)

# below I am only going to run metrics on the first five opsims
for run in dbRuns[0:5]:
    # must set run name for each opSim to store metric data into
    # separate files
    maxairmassSky.setRunName(run)
    CoaddY.setRunName(run)
    CoaddU.setRunName(run)
    metricGroup = metricBundles.MetricBundleGroup(bundleDict,\
                    opSimDbs[run], metricDataPath, resultDbs[run])
    metricGroup.runAll()

Querying table None with constraint filter = "u" and note not like "DD%" for columns ['fieldDec', 'rotSkyPos', 'fieldRA', 'fiveSigmaDepth']
Found 130488 visits
Running:  ['CoaddU']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying table None with constraint filter = "g" and note not like "DD%" for columns ['fieldDec', 'rotSkyPos', 'airmass', 'fieldRA']
Found 198940 visits
Running:  ['maxairmassSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying table None with constraint filter = "y" and note not like "DD%" for columns ['fieldDec', 'rotSkyPos', 'fieldRA', 'fiveSigmaDepth']
Found 380658 visits
Running:  ['CoaddY']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying table None with constraint filter = "u" and note not like "DD%" for columns ['fieldDec', 'rotSkyPos', 'fieldRA', 'fiveSigmaDepth']
Found 128883 visits
Running:  ['CoaddU']


Now you have computed your metric for all the opSims.  To look at those results, go through the  [View_Results.ipynb](./02_View_Results.ipynb) notebook.